# Booth_Multiplication
# Booth_Multiplication
© 2024 Brian Butka. All rights reserved.

<a href="https://colab.research.google.com/github/bbutka/CEC220/blob/main/Booth_Multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
from termcolor import colored

def binary_to_decimal(binary_str):
    return int(binary_str, 2) if binary_str[0] == '0' else int(binary_str, 2) - (1 << len(binary_str))

def decimal_to_binary(n, bits):
    if n >= 0:
        s = bin(n & int("1"*bits, 2))[2:]
    else:
        s = bin((1 << bits) + n)[2:]
    return ("{0:0>%s}" % (bits)).format(s)

def booth_transition(prev, curr):
    if prev == '0' and curr == '0':
        return ' 0'
    elif prev == '0' and curr == '1':
        return ' 1'
    elif prev == '1' and curr == '1':
        return ' 0'
    elif prev == '1' and curr == '0':
        return '-1'

def apply_transition_chart(binary_str):
    extended_binary = f'0{binary_str}0'
    transitions = []

    for i in range(len(extended_binary) - 1):
        transition_value = booth_transition(extended_binary[i], extended_binary[i+1])
        transitions.append(transition_value)

    return extended_binary, transitions

def binary_addition_with_carry(a, b):
    max_len = max(len(a), len(b))
    a = a.zfill(max_len)
    b = b.zfill(max_len)

    carry = 0
    result = ''
    carries = ''

    for i in range(max_len - 1, -1, -1):
        total = int(a[i]) + int(b[i]) + carry
        if total > 1:
            carry = 1
            carries = '1' + carries
        else:
            carry = 0
            carries = ' ' + carries  # Change '0' to ' ' for blank carry
        result = str(total % 2) + result

    # Shift carries one position to the left and add a trailing space
    carries = carries[1:] + ' '
    return result.zfill(max_len), carries.zfill(max_len)

def sign_extend(value, bits):
    if value[0] == '0':
        return value.zfill(bits)
    else:
        return value.rjust(bits, '1')

def binary_multiplication(M, Q):
    N = len(Q) // 2  # Calculate number of pairs
    M = M.zfill(N)
    partial_products = []

    M_dec = binary_to_decimal(M)
    neg_M = decimal_to_binary(-M_dec, N)

    # Calculate partial products
    for i in range(N):
        pair = Q[2 * (N - 1 - i):2 * (N - i)]
        if pair == ' 1':
            partial_product = M + '0' * i
        elif pair == '-1':
            partial_product = neg_M + '0' * i
        else:
            partial_product = '0' * (N + i)
        partial_products.append(sign_extend(partial_product, 2 * N))

    # Initialize final product and carries
    final_product = '0' * (2 * N)
    total_carries = ' ' * (2 * N)

    # Sum the partial products to get the final product
    for partial_product in partial_products:
        final_product, carries = binary_addition_with_carry(final_product, partial_product)
        total_carries, _ = binary_addition_with_carry(total_carries.replace(' ', '0'), carries.replace(' ', '0'))

    final_product_dec = binary_to_decimal(final_product)
    return M, Q, partial_products, final_product, final_product_dec, total_carries

def main():
    binary_str = input("Enter a binary number: ").strip()
    decimal_input = binary_to_decimal(binary_str)
    extended_binary, transitions = apply_transition_chart(binary_str)

    print(f"Input Binary: {binary_str} (Decimal: {decimal_input})")
    print(f"Extended Binary with 0's: {colored('0', 'red')}{binary_str}{colored('0', 'red')}")

    print("\nTransition Values for Each Pair of Bits:")
    pairs = [f"{extended_binary[i]}{extended_binary[i+1]}" for i in range(len(extended_binary) - 1)]
    print("Pairs: ", " -> ".join(pairs))
    print("Values:", " -> ".join(map(str, transitions)))
    print("       ", "    ".join(map(str, transitions)).lstrip('0'))  # Printing the values without arrows below the pairs line, dropping leading zeros

    result_decimal = sum((1 if val == ' 1' else -1 if val == '-1' else 0) * (2 ** i) for i, val in enumerate(reversed(transitions)))
    print(f"\nResult in Decimal: {result_decimal}")

    multiply = input("Do you wish to multiply this number by another binary number? (yes/no): ").strip().lower()

    if multiply == 'yes':
        binary_Q = "".join(transitions)  # Assign transitions line to Q
        binary_M = input("Enter the binary number to multiply with: ").strip()
        M, Q, partial_products, final_product_bin, final_product_dec, total_carries = binary_multiplication(binary_M, binary_Q)

        M_dec = binary_to_decimal(M)
        Q_dec = result_decimal  # Use the previously calculated result as decimal value for Q
        neg_M_binary = decimal_to_binary(-M_dec, len(M))  # Calculate binary representation of -M

        N = len(Q) // 2  # Ensure N is properly defined

        t = 6  # Specify the number of columns to move to the right

        # Function to expand digits with spaces
        def expand_digits(binary_str):
            return " ".join([c for c in binary_str])

        # Function to expand transitions
        def expand_transitions(transitions):
            return " ".join(['-1' if transitions[i:i+2] == '-1' else ' 0' if transitions[i:i+2] == ' 0' else ' 1' for i in range(0, len(transitions), 2)])

        # Calculate the required width for alignment
        max_len = 2 * N + (N - 1)
        M_expanded = expand_digits(M).rjust(max_len)
        neg_M_expanded = expand_digits(neg_M_binary).rjust(max_len)
        Q_expanded = expand_transitions(Q).rjust(max_len)

        # Display the multiplication problem
        print(f"\nMultiplication Problem:")
        print(f"{' ' * (t+6)}{M_expanded} (M = {M_dec}, -M = {neg_M_expanded})")
        print(f"{' ' * (t+5)}x {Q_expanded} (Q = {Q_dec})")
        print(f"{' ' * (t+6)}{'-' * max_len}")

        # Display the carries shifted one character to the left
        print(f"{' ' * t}{expand_digits(total_carries).rjust(max_len)} (carries)")

        # Display the partial products
        for i, partial_product in enumerate(partial_products):
            print(f"{' ' * t}{expand_digits(partial_product).rjust(max_len)} (partial product {i+1})")

        # Display the final result
        print(f"{' ' * t}{'-' * max_len}")
        print(f"{' ' * t}{expand_digits(final_product_bin).rjust(max_len)} (final product)")
        print(f"{' ' * t}Final product in decimal: {final_product_dec}")

if __name__ == "__main__":
    main()


Enter a binary number: 01111
Input Binary: 01111 (Decimal: 15)
Extended Binary with 0's: 0011110

Transition Values for Each Pair of Bits:
Pairs:  00 -> 01 -> 11 -> 11 -> 11 -> 10
Values:  0 ->  1 ->  0 ->  0 ->  0 -> -1
         0     1     0     0     0    -1

Result in Decimal: 15
Do you wish to multiply this number by another binary number? (yes/no): yes
Enter the binary number to multiply with: 01010

Multiplication Problem:
                  0 0 1 0 1 0 (M = 10, -M =       1 1 0 1 1 0)
           x  0  1  0  0  0 -1 (Q = 15)
            -----------------
      1 1 1 1 1 1 0 0 0 0 0 0 (carries)
      1 1 1 1 1 1 1 1 0 1 1 0 (partial product 1)
      0 0 0 0 0 0 0 0 0 0 0 0 (partial product 2)
      0 0 0 0 0 0 0 0 0 0 0 0 (partial product 3)
      0 0 0 0 0 0 0 0 0 0 0 0 (partial product 4)
      0 0 0 0 1 0 1 0 0 0 0 0 (partial product 5)
      0 0 0 0 0 0 0 0 0 0 0 0 (partial product 6)
      -----------------
      0 0 0 0 1 0 0 1 0 1 1 0 (final product)
      Final product in 